### Structured output
#### Models can be requested to provide their response in a format matching a given schema. This is useful for ensuring the output can be easily parsed and used in subsequent processing. LangChain supports multiple schema types and methods for enforcing structured output.

### Pydantic
##### Pydantic models provide the richest feature set with field validation, descriptions, and nested structures.

In [1]:
import os
from langchain.chat_models import init_chat_model
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model=init_chat_model("groq:qwen/qwen3-32b")
model

c:\Users\ujami\Downloads\PHD_thesis\langchain_learning\.venv\Lib\site-packages\langchain_core\_api\deprecation.py:26: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000019F9696D6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019F9696E120>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [2]:
from pydantic import BaseModel,Field

class Movie(BaseModel):
    title:str=Field(description="The title of the movie")
    year:int=Field(description="This year the movie was released")
    director:str=Field(description="The director of the movie")
    rating:float=Field(description="The movies rating out of 10")

In [3]:
model_with_structure=model.with_structured_output(Movie)
model_with_structure

RunnableBinding(bound=ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 16384, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x0000019F9696D6A0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000019F9696E120>, model_name='qwen/qwen3-32b', model_kwargs={}, groq_api_key=SecretStr('**********')), kwargs={'tools': [{'type': 'function', 'function': {'name': 'Movie', 'description': '', 'parameters': {'properties': {'title': {'description': 'The title of the movie', 'type': 'string'}, 'year': {'description': 'This year the movie was released', 'type': 'integer'}, 'director': {'description': 'The director of the movie', 'type': 'string'}, 'rating': {'description': 'The movies rating out of 10', 'type': 'number'}}, 'required': ['title', 'year', '

In [5]:
model_with_structure.invoke("Provide details about the moview Inception")

Movie(title='Inception', year=2010, director='Christopher Nolan', rating=8.8)

In [4]:
model.invoke("Provide details about the moview Inception")

AIMessage(content='<think>\nOkay, so I need to provide details about the movie Inception. Let me start by remembering what I know about it. It\'s a science fiction action film directed by Christopher Nolan, right? He did other popular movies like The Dark Knight and Interstellar. The main actor is Leonardo DiCaprio, and he plays the lead role. The title is Inception, which I think has something to do with entering someone\'s dreams. \n\nI recall that the plot involves a group of people who can enter people\'s dreams to steal or implant ideas. The main character, whose name is Cobb, is a thief who does this for a living. His team is trying to break into someone\'s subconscious to steal information, but they get the chance to do the opposite, which is to implant an idea, which is called incepting. \n\nThe movie\'s themes might include reality versus dreams, time, and the nature of the mind. There\'s a lot of action in the dream sequences, with different layers of dreams affecting the tim

##### MEssage output alongside parsed structure

In [6]:
from pydantic import BaseModel, Field

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie, include_raw=True)  

response = model_with_structure.invoke("Provide details about the movie Inception")
response

{'raw': AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for details about the movie Inception. Let me see what I need to do. The tools provided include a Movie function that requires title, year, director, and rating. I need to fill in those parameters. \n\nFirst, I know the title is "Inception". The year it was released was 2010. The director is Christopher Nolan. As for the rating, I think it\'s around 8.8 on IMDb. Let me double-check those facts to make sure. Yep, that\'s correct. So I\'ll structure the tool call with those details. Need to make sure the JSON is formatted correctly with the required fields. No need for any extra information since the user just wants the details provided by the function.\n', 'tool_calls': [{'id': 'va0shz686', 'function': {'arguments': '{"director":"Christopher Nolan","rating":8.8,"title":"Inception","year":2010}', 'name': 'Movie'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 2

### Nested Structure

In [8]:
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI

class Actor(BaseModel):
    name: str
    role: str

class MovieDetails(BaseModel):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model = ChatOpenAI(model="gpt-4o-mini")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke(
    "Provide details about the movie Inception"
)

print(response)


title='Inception' year=2010 cast=[Actor(name='Leonardo DiCaprio', role='Dom Cobb'), Actor(name='Joseph Gordon-Levitt', role='Arthur'), Actor(name='Elliot Page', role='Ariadne'), Actor(name='Tom Hardy', role='Eames'), Actor(name='Ken Watanabe', role='Saito'), Actor(name='Cillian Murphy', role='Robert Fischer'), Actor(name='Marion Cotillard', role='Mal'), Actor(name='Michael Caine', role='Professor Stephen Miles')] genres=['Action', 'Adventure', 'Sci-Fi', 'Thriller'] budget=160.0


#### TypedDict

TypedDict provides a simpler alternative using Python’s built-in typing, ideal when you don’t need runtime validation.

In [9]:
from typing_extensions import TypedDict,Annotated

class MovieDict(TypedDict):
    """A movie with details."""
    title: Annotated[str, ..., "The title of the movie"]
    year: Annotated[int, ..., "The year the movie was released"]
    director: Annotated[str, ..., "The director of the movie"]
    rating: Annotated[float, ..., "The movie's rating out of 10"]


model_withtypedict=model.with_structured_output(MovieDict)
response=model_withtypedict.invoke("Please provide the details of the movie avengers")
response

{'title': 'The Avengers',
 'year': 2012,
 'director': 'Joss Whedon',
 'rating': 8.0}

In [10]:
class Actor(TypedDict):
    name: str
    role: str

class MovieDetails(TypedDict):
    title: str
    year: int
    cast: list[Actor]
    genres: list[str]
    budget: float | None = Field(None, description="Budget in millions USD")

model_with_structure = model.with_structured_output(MovieDetails)

response = model_with_structure.invoke("Provide details about the movie Avengers")
response

{'title': 'The Avengers',
 'year': 2012,
 'cast': [{'name': 'Robert Downey Jr.', 'role': 'Tony Stark / Iron Man'},
  {'name': 'Chris Evans', 'role': 'Steve Rogers / Captain America'},
  {'name': 'Scarlett Johansson', 'role': 'Natasha Romanoff / Black Widow'},
  {'name': 'Chris Hemsworth', 'role': 'Thor'},
  {'name': 'Mark Ruffalo', 'role': 'Bruce Banner / Hulk'},
  {'name': 'Jeremy Renner', 'role': 'Clint Barton / Hawkeye'},
  {'name': 'Tom Hiddleston', 'role': 'Loki'}],
 'genres': ['Action', 'Adventure', 'Sci-Fi'],
 'budget': 220000000}

In [11]:
model.profile

{'max_input_tokens': 128000,
 'max_output_tokens': 16384,
 'image_inputs': True,
 'audio_inputs': False,
 'video_inputs': False,
 'image_outputs': False,
 'audio_outputs': False,
 'video_outputs': False,
 'reasoning_output': False,
 'tool_calling': True,
 'structured_output': True,
 'image_url_inputs': True,
 'pdf_inputs': True,
 'pdf_tool_message': True,
 'image_tool_message': True,
 'tool_choice': True}

#### DataClasses

A data class is a class typically containing mainly data, although there aren’t really any restrictions. You create it using the @dataclass decorator

In [13]:
import os
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")

In [16]:
from pydantic import BaseModel, Field
from langchain.agents import create_agent


class ContactInfo(BaseModel):
    """Contact information for a person."""
    name: str = Field(description="The name of the person")
    email: str = Field(description="The email address of the person")
    phone: str = Field(description="The phone number of the person")


agent = create_agent(
    model="gpt-4o-mini",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result

{'messages': [HumanMessage(content='Extract contact info from: John Doe, john@example.com, (555) 123-4567', additional_kwargs={}, response_metadata={}, id='c3698de8-d8b0-4de2-b263-72af424254e9'),
  AIMessage(content='{"name":"John Doe","email":"john@example.com","phone":"(555) 123-4567"}', additional_kwargs={'parsed': None, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 125, 'total_tokens': 153, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_29330a9688', 'id': 'chatcmpl-CrSixLJL49vuxm3ENi7ilh9X85w7w', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b60ef-023b-79b0-a4d8-8e28e1696d2f-0', usage_metadata={'input_tokens': 125, 'output_tokens': 28, 'total_tok

In [17]:
result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')

In [18]:
## Typedict
from typing_extensions import TypedDict
from langchain.agents import create_agent


class ContactInfo(TypedDict):
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person

agent = create_agent(
    model="gpt-4o-mini",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

{'name': 'John Doe', 'email': 'john@example.com', 'phone': '(555) 123-4567'}

In [20]:
## Dataclass

from dataclasses import dataclass
from langchain.agents import create_agent

@dataclass
class ContactInfo:
    """Contact information for a person."""
    name: str # The name of the person
    email: str # The email address of the person
    phone: str # The phone number of the person


agent = create_agent(
    model="gpt-4o-mini",
    response_format=ContactInfo  # Auto-selects ProviderStrategy
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

result["structured_response"]

ContactInfo(name='John Doe', email='john@example.com', phone='(555) 123-4567')